Generally following:<br>
https://medium.com/data-from-the-trenches/text-classification-the-first-step-toward-nlp-mastery-f5f95d525d73

In [1]:
import pprint
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

In [2]:
file00 = 'C:\\UT\\DataMining\\books\\set0\\horror00.txt'
file01 = 'C:\\UT\\DataMining\\books\\set0\\horror01.txt'
file02 = 'C:\\UT\\DataMining\\books\\set0\\horror02.txt'
file03 = 'C:\\UT\\DataMining\\books\\set0\\horror03.txt'
file04 = 'C:\\UT\\DataMining\\books\\set0\\horror04.txt'
file05 = 'C:\\UT\\DataMining\\books\\set0\\horror05.txt'
file06 = 'C:\\UT\\DataMining\\books\\set0\\horror06.txt'
file07 = 'C:\\UT\\DataMining\\books\\set0\\horror07.txt'
file08 = 'C:\\UT\\DataMining\\books\\set0\\horror08.txt'
file09 = 'C:\\UT\\DataMining\\books\\set0\\horror09.txt'
file10 = 'C:\\UT\\DataMining\\books\\set0\\humor00.txt'
file11 = 'C:\\UT\\DataMining\\books\\set0\\humor01.txt'
file12 = 'C:\\UT\\DataMining\\books\\set0\\humor02.txt'
file13 = 'C:\\UT\\DataMining\\books\\set0\\humor03.txt'
file14 = 'C:\\UT\\DataMining\\books\\set0\\humor04.txt'
file15 = 'C:\\UT\\DataMining\\books\\set0\\humor05.txt'
file16 = 'C:\\UT\\DataMining\\books\\set0\\humor06.txt'
file17 = 'C:\\UT\\DataMining\\books\\set0\\humor07.txt'
file18 = 'C:\\UT\\DataMining\\books\\set0\\humor08.txt'
file19 = 'C:\\UT\\DataMining\\books\\set0\\humor09.txt'
train_files = [file00, file01, file02, file03, file04, 
               file05, file06, file07, file08, file09,
               file10, file11, file12, file13, file14,
               file15, file16, file17, file18, file19]

In [3]:
file00 = 'C:\\UT\\DataMining\\books\\set0\\horror10.txt'
file01 = 'C:\\UT\\DataMining\\books\\set0\\horror11.txt'
file02 = 'C:\\UT\\DataMining\\books\\set0\\horror12.txt'
file03 = 'C:\\UT\\DataMining\\books\\set0\\horror13.txt'
file04 = 'C:\\UT\\DataMining\\books\\set0\\horror14.txt'
file05 = 'C:\\UT\\DataMining\\books\\set0\\horror15.txt'
file06 = 'C:\\UT\\DataMining\\books\\set0\\horror16.txt'
file07 = 'C:\\UT\\DataMining\\books\\set0\\horror17.txt'
file08 = 'C:\\UT\\DataMining\\books\\set0\\horror18.txt'
file09 = 'C:\\UT\\DataMining\\books\\set0\\horror19.txt'
file10 = 'C:\\UT\\DataMining\\books\\set0\\humor10.txt'
file11 = 'C:\\UT\\DataMining\\books\\set0\\humor11.txt'
file12 = 'C:\\UT\\DataMining\\books\\set0\\humor12.txt'
file13 = 'C:\\UT\\DataMining\\books\\set0\\humor13.txt'
file14 = 'C:\\UT\\DataMining\\books\\set0\\humor14.txt'
file15 = 'C:\\UT\\DataMining\\books\\set0\\humor15.txt'
file16 = 'C:\\UT\\DataMining\\books\\set0\\humor16.txt'
file17 = 'C:\\UT\\DataMining\\books\\set0\\humor17.txt'
file18 = 'C:\\UT\\DataMining\\books\\set0\\humor18.txt'
file19 = 'C:\\UT\\DataMining\\books\\set0\\humor19.txt'
test_files = [file00, file01, file02, file03, file04, 
              file05, file06, file07, file08, file09,
              file10, file11, file12, file13, file14,
              file15, file16, file17, file18, file19
             ]

In [4]:
train_texts = []
for file in train_files:
    with open(file, 'r', encoding='utf-8') as f:
        text = f.read()
        train_texts.append(text)

In [5]:
# Category 0 == Horror
# Category 1 == Humor
category = [0, 0, 0, 0, 0,
            0, 0, 0, 0, 0,
            1, 1, 1, 1, 1,
            1, 1, 1, 1, 1,]

In [6]:
df_train = pd.DataFrame(np.array(train_texts))
df_train['Category'] = category
df_train.columns = ["text", "category"]

In [7]:
df_train.head()

,text,category
0,﻿The Project Gutenberg EBook of The Damned Thi...,0
1,﻿Project Gutenberg's The Fall of the House of ...,0
2,"﻿The Project Gutenberg eBook of The Beetle, by...",0
3,"﻿The Project Gutenberg EBook of Dracula, by Br...",0
4,"﻿The Project Gutenberg eBook of Frankenstein, ...",0


In [8]:
test_texts = []
for file in test_files:
    with open(file, 'r', encoding='utf-8') as f:
        text = f.read()
        test_texts.append(text)

In [9]:
df_test = pd.DataFrame(np.array(test_texts))
df_test['Category'] = category
df_test.columns = ["text", "category"]

In [10]:
df_test.head()

,text,category
0,"﻿The Project Gutenberg EBook of The Wendigo, b...",0
1,﻿The Project Gutenberg EBook of The Dunwich Ho...,0
2,﻿The Project Gutenberg EBook of The Great God ...,0
3,﻿The Project Gutenberg eBook of The Monk: a ro...,0
4,﻿The Project Gutenberg eBook of Flowering Evil...,0


In [11]:
def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)
    
    # remove numbers including unsigned / signed integers and floats
    text = re.sub(r"[0-9]+", "", text)
    text = re.sub(r"[-0-9]+", "", text)
    text = re.sub(r"[+.0-9]+", "", text)
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    
    # remove non-ascii characters
    text = re.sub(r'[^\x00-\x7F]+','', text)

    return text

In [ ]:
#returns a list of words after cleaning
#data0 = clean_text(data).split()

In [ ]:
#len(data0)

In [ ]:
# training_texts = [
#     "This is a good cat",
#     "This is a bad day"
# ]

# test_texts = [
#     "This day is a good day"
# ]

# # this vectorizer will skip stop words
# vectorizer = CountVectorizer(
#     stop_words="english",
#     preprocessor=clean_text
# )

# # fit the vectorizer on the training text
# vectorizer.fit(training_texts)

# # get the vectorizer's vocabulary
# inv_vocab = {v: k for k, v in vectorizer.vocabulary_.items()}
# vocabulary = [inv_vocab[i] for i in range(len(inv_vocab))]

# # vectorization example
# pd.DataFrame(
#     data=vectorizer.transform(test_texts).toarray(),
#     index=["test sentence"],
#     columns=vocabulary
# )

In [ ]:
# # this vectorizer will skip stop words
# vectorizer = CountVectorizer(
#     stop_words="english",
#     preprocessor=clean_text,
#     strip_accents="ascii"
# )

# # fit the vectorizer on the training text
# vectorizer.fit(training_texts)

# # get the vectorizer's vocabulary
# inv_vocab = {v: k for k, v in vectorizer.vocabulary_.items()}
# vocabulary = [inv_vocab[i] for i in range(len(inv_vocab))]


In [ ]:
# # this vectorizer will skip stop words
# vectorizer = CountVectorizer(
#     stop_words="english",
#     preprocessor=clean_text,
#     strip_accents="ascii"
# )

# # fit the vectorizer on the training text
# X = vectorizer.fit_transform(training_texts)

# # get the vectorizer's vocabulary
# #print(vectorizer.get_feature_names())


In [ ]:
#print(X.toarray())

In [ ]:
#len(vocabulary)

In [12]:
# Transform each text into a vector of word counts
vectorizer = CountVectorizer(stop_words="english",
                             preprocessor=clean_text,
                             strip_accents="ascii")

train_features = vectorizer.fit_transform(df_train["text"])    
test_features = vectorizer.transform(df_test["text"])

# Training
model = LinearSVC()
model.fit(train_features, df_train["category"])
y_pred = model.predict(test_features)

# Evaluation
acc = accuracy_score(df_test["category"], y_pred)

print("Accuracy on the IMDB dataset: {:.2f}".format(acc*100))

Accuracy on the IMDB dataset: 80.00


In [14]:
# Transform each text into a vector of word counts
vectorizer = TfidfVectorizer(stop_words="english",
                             preprocessor=clean_text,
                             ngram_range=(1, 2))

train_features = vectorizer.fit_transform(df_train["text"])
test_features = vectorizer.transform(df_test["text"])

# Training
model = LinearSVC()
model.fit(train_features, df_train["category"])
y_pred = model.predict(test_features)

# Evaluation
acc = accuracy_score(df_test["category"], y_pred)

print("Accuracy on the IMDB dataset: {:.2f}".format(acc*100))

Accuracy on the IMDB dataset: 80.00
